# Units and coordinates

`cosipy` uses astropy's units and coordinates. 

## SpacecraftFrame

SkyCoord objects are instantiated by passing a location with specified units and a coordinate frame. `cosipy` defines the custom coordinate system `SpacecraftFrame` that uses the spacecraft as the frame of reference. 

To create a coordinate with this frame you can simply do:

In [1]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from cosipy.coordinates import SpacecraftFrame

c = SkyCoord(lon = 45*u.deg, lat = 10*u.deg, frame = SpacecraftFrame())

This allows you to know the reference frame of the coordinate, e.g.

In [2]:
c.frame

<SpacecraftFrame Coordinate (attitude=None, obstime=None, location=None): (lon, lat) in deg
    (45., 10.)>

However, in order to transform it to other coordinates you need to specify the orientation of the spacecraft with respect to the ICRS reference frame --i.e. the attitude. 

<div class="alert alert-info">
DEVELOPERS: During flight I'd expect to recieve a quaternion as part of the telemetry. This is used by GBM, for example, since it is more compact than a rotation matrix and behaves better than Euler rotation angles.
</div>

In [3]:
from cosipy.coordinates import Attitude

attitude = Attitude.from_rotvec(45*u.deg*[0,1,0])

c = SkyCoord(lon = 45*u.deg, lat = 10*u.deg, frame = SpacecraftFrame(attitude = attitude))

Once this is specified, we can transform from/to any other frame supported by astropy

In [4]:
c.transform_to('icrs')

<SkyCoord (ICRS): (ra, dec) in deg
    (48.54153446, -21.69194144)>

In [5]:
c.transform_to('galactic')

<SkyCoord (Galactic): (l, b) in deg
    (211.32571205, -57.16686803)>

In [6]:
c = SkyCoord(ra = 50*u.deg, dec = 5*u.deg)

c.transform_to(SpacecraftFrame(attitude = attitude))

<SkyCoord (SpacecraftFrame: attitude=<quat = [0.         0.38268343 0.         0.92387953], frame = icrs>, obstime=None, location=None): (lon, lat) in deg
    (62.86143232, 30.95858198)>

Although it does not play a role in this particular coordinates transformation, the observation time and location with respect of the Earth can also be specified in case it is needed by any other algorithm:

In [7]:
from astropy.time import Time
from astropy.coordinates import EarthLocation

frame = SpacecraftFrame(attitude = attitude,
                        obstime = Time('2026-01-01T00:00:00'),
                        location = EarthLocation(lon = 10*u.deg, lat = 46*u.deg, height = 400*u.km))

c = SkyCoord(lon = 45*u.deg, lat = 10*u.deg, frame = frame)

In [8]:
c.frame.obstime

<Time object: scale='utc' format='isot' value=2026-01-01T00:00:00.000>